## Getting the evidence for an RS ID

Suppose you have an RS ID (e.g. `rs379920406`) for a variant and you're interested in finding what evidence for that variant is contained in the EVA. Specifically you want to know the allele frequencies and genotypes present in any study containing that variant. This notebook will walk you through how to do that using EVA APIs.

For reference, here is the Swagger documentation for all the APIs we will use:

* [Identifiers API](https://www.ebi.ac.uk/eva/webservices/identifiers/swagger-ui.html)
* [Contig Alias API](https://www.ebi.ac.uk/eva/webservices/contig-alias/swagger-ui.html)
* [Variant Info API](https://www.ebi.ac.uk/eva/webservices/rest/swagger-ui.html)

In [1]:
import requests
from collections import Counter

In [2]:
# The variant ID in question - you can change this and everything else should still run
rsid = 379920406

In [3]:
identifiers_url = f'https://www.ebi.ac.uk/eva/webservices/identifiers/v1/clustered-variants/{rsid}/submitted'

response = requests.get(identifiers_url)
submitted_variants = response.json()
variants_with_evidence = [sv for sv in submitted_variants if sv['data']['supportedByEvidence']]

This API endpoint responds with a list of submitted variants associated with the provided clustered variant ID. We've filtered this response to include only those supported by evidence in the EVA (genotypes and allele frequencies).

In [4]:
variants_with_evidence[0]

{'accession': 7571130482,
 'version': 1,
 'data': {'referenceSequenceAccession': 'GCA_002263795.2',
  'taxonomyAccession': 9913,
  'projectAccession': 'PRJEB46861',
  'contig': 'CM008173.2',
  'start': 85411136,
  'referenceAllele': 'C',
  'alternateAllele': 'T',
  'clusteredVariantAccession': 379920406,
  'supportedByEvidence': True,
  'assemblyMatch': True,
  'allelesMatch': True,
  'validated': False,
  'mapWeight': None,
  'createdDate': '2021-09-08T13:08:25.551',
  'remappedFrom': None,
  'remappedDate': None,
  'remappingId': None,
  'backPropagatedVariantAccession': None}}

The actual genotypes and allele frequencies aren't present in this response, but we can get those from another API endpoint which will return more detailed variant information.

There are some quirks to this process which we're working to improve, but here's how you can do it currently.

In [5]:
# Get relevant accessions
assembly_accession = variants_with_evidence[0]['data']['referenceSequenceAccession']
taxonomy_id = variants_with_evidence[0]['data']['taxonomyAccession']
contig_accession = variants_with_evidence[0]['data']['contig']

In [6]:
# Translate the accessions to names - optional if you know the names already
species_list_url = 'https://www.ebi.ac.uk/eva/webservices/rest/v1/meta/species/list'
response = requests.get(species_list_url)
species_list = response.json()['response'][0]['result']
for s in species_list:
    if s['assemblyAccession'] == assembly_accession and s['taxonomyId'] == taxonomy_id:
        assembly_code = s['assemblyCode']
        taxonomy_code = s['taxonomyCode']
        break
        
contig_alias_url = f'https://www.ebi.ac.uk/eva/webservices/contig-alias/v1/chromosomes/genbank/{contig_accession}'
response = requests.get(contig_alias_url)
chromosomes = response.json()['_embedded']['chromosomeEntities']
for c in chromosomes:
    if c['assembly']['insdcAccession'] == assembly_accession:
        chrom = c['enaSequenceName']
        break

In [7]:
print(f'{assembly_accession}\t=> {assembly_code}')
print(f'{taxonomy_id}\t\t=> {taxonomy_code}')
print(f'{contig_accession}\t=> {chrom}')

GCA_002263795.2	=> arsucd12
9913		=> btaurus
CM008173.2	=> 6


In [8]:
# Position and reference allele should be the same for all, but the alternate allele may be different,
# so we collect all of them to query
alternate_alleles = {v['data']['alternateAllele'] for v in variants_with_evidence}
pos = variants_with_evidence[0]['data']['start']
ref = variants_with_evidence[0]['data']['referenceAllele']

source_variants = []
for alt in alternate_alleles:
    variant_info_url = f'https://www.ebi.ac.uk/eva/webservices/rest/v2/variants/{chrom}:{pos}:{ref}:{alt}/sources?' \
                       f'species={taxonomy_code}&assembly={assembly_code}'
    response = requests.get(variant_info_url)
    source_variants.extend(response.json()['_embedded']['variantSourceEntryWithSampleNamesList'])

This API endpoint returns the full submitted variant details stored in the EVA, including sample data. It can be very big which is why we don't include it every time we return a variant!

In [9]:
source_variants[0]

{'fileId': 'ERZ2968700',
 'studyId': 'PRJEB46861',
 'format': 'GT',
 'cohortStats': {'ALL': {'refAllele': 'C',
   'altAllele': 'T',
   'variantType': 'SNV',
   'refAlleleCount': 0,
   'altAlleleCount': 0,
   'missingAlleles': 832,
   'missingGenotypes': 416,
   'refAlleleFreq': 0.0,
   'altAlleleFreq': 0.0,
   'maf': 0.1521739,
   'mgf': 0.0,
   'mafAllele': 'T',
   'mgfGenotype': '1/1',
   'numSamples': 0,
   'transition': True,
   'transversion': False}},
 'attributes': {'CSQ': 'T|5_prime_UTR_variant|MODIFIER|CSN1S1|ENSBTAG00000007695|Transcript|ENSBTAT00000010119|protein_coding|1/19||ENSBTAT00000010119.3:c.-682C>T||19|||||rs379920406||1||SNV|VGNC||YES|||P5||ENSBTAP00000010119|P02662||UPI0000126FC7||||||||||||||||||||||||||||||||,T|5_prime_UTR_variant|MODIFIER|CSN1S1|ENSBTAG00000007695|Transcript|ENSBTAT00000073072|protein_coding|1/17||ENSBTAT00000073072.1:c.-682C>T||19|||||rs379920406||1||SNV|VGNC|||||A2||ENSBTAP00000069331|||UPI000383DA63||||||||||||||||||||||||||||||||,T|5_prime_U

In [10]:
# Get the minor allele frequencies
allele_freqs = []
for variant in source_variants:
    if 'cohortStats' in variant:
        allele_freqs.append(variant['cohortStats']['ALL']['maf'])

In [11]:
allele_freqs

[0.1521739, 0.111]

In [12]:
# Get all the genotypes present in samples
genotypes = Counter()
for variant in source_variants:
    if 'samplesData' in variant:
        for _, sample in variant['samplesData'].items():
            if 'GT' in sample:
                genotypes[sample['GT']] += 1

In [13]:
# Note that -1 means the value is missing
for gt, count in genotypes.items():
    print(f'{gt}: {count}')

0/1: 14
0/0: 32
-1/-1: 416
